In [81]:
from skimage import data, draw,color, data, restoration
from skimage import transform, util
from scipy import misc, fftpack, signal
import numpy as np
from skimage import filters, color
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import matplotlib.animation as animation
from scipy.misc import imresize
from scipy import ndimage as nd
import scipy

In [97]:
rgb =  plt.imread('casa.jpg')
img_gris = color.rgb2gray(rgb)
print img_gris.shape
print rgb.shape
fig = plt.figure(121) 
plt.imshow(rgb)
plt.title('Original Image')
fig = plt.figure(122) 
plt.imshow(img_gris,cmap="gray")
plt.title('Gray image')
plt.show()

(480, 640)
(480, 640, 3)


In [98]:
def gradient(img):
    img = color.rgb2gray(img)
    gx, gy = np.gradient(img)
    grad = np.sqrt(gx*gx + gy*gy)
    return grad
grad = gradient(img)
#normalitzacio d'imatge
def normalitzar(imatge):
    return np.real((imatge - np.min(imatge)) * 256 / (np.max(imatge) - np.min(imatge)))

In [99]:
#normalitzo les imatges a 255
rgb=normalitzar(rgb)
img_gris=normalitzar(img_gris)
img_grad=gradient(rgb)
img_grad=normalitzar(img_grad)
print img_grad.shape
plt.imshow(rgb.astype('uint8'))
plt.title('Original Image')
fig = plt.figure(122) 
plt.imshow(img_gris.astype('uint8'),cmap="gray")
plt.title('Gray image')
fig = plt.figure(123) 
plt.imshow(img_grad.astype('uint8'),cmap="gray")
plt.title('Contorn')
plt.show()

(480, 640)


In [100]:
class get_mouse_click():
    
    #Mouse interaction interface for radial distortion removal.
    def __init__(self, img):
      height, width = img.shape[:2]
      self.figure = plt.imshow(img, extent=(0, width, height, 0))
      plt.gray()
      plt.title('select the object to remove')
      plt.xlabel('Select sets of  points with left mouse button,\n'
                 'click right button to close the polygon.')
      plt.connect('button_press_event', self.button_press)
      plt.connect('motion_notify_event', self.mouse_move)

      self.img = np.atleast_3d(img)
      self.points = []
      self.centre = np.array([(width - 1)/2., (height - 1)/2.])

      self.height = height
      self.width = width

      self.make_cursorline()
      self.figure.axes.set_autoscale_on(False)

      plt.show()
      plt.close()

    def make_cursorline(self):
        self.cursorline, = plt.plot([0],[0],'r:+', linewidth=2,markersize=15,markeredgecolor='b')

    def button_press(self,event):
        # Register mouse clicks.
        if (event.button == 1 and event.xdata and event.ydata):
            self.points.append((event.xdata,event.ydata))
            plt.gca().lines.append(self.cursorline)
            plt.gca()
            self.make_cursorline()
        if (event.button != 1):
            self.points.append((self.points[0][0],self.points[0][1]))
            plt.close()
            return self.points

    def mouse_move(self,event):
        # Handle cursor drawing.
        pts_last_set=len(self.points)
        pts = np.zeros((pts_last_set+1,2))
        if pts_last_set > 0:
            # Line follows up to 3 clicked points:
            pts[:pts_last_set] = self.points[-pts_last_set:]
            # The last point of the line follows the mouse cursor
        pts[pts_last_set:] = [event.xdata,event.ydata]
        self.cursorline.set_data(pts[:,0], pts[:,1])
        plt.draw()
def compute_mask(width,height,polygon):
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    #mask100 = 100*np.ones([mask.shape[0],mask.shape[1]])
    #mask=mask*101
    #mask = 1. - mask; # switch 0s and 1s
    return mask
def convine_mask(zona_delete,img_gris):
    #si tenim 1 a la macara tenim forat
    for i in xrange(len(img_gris)):
        for j in xrange(len(img_gris[0])):
            if zona_delete[i][j]==1:
                img_gris[i][j]=-100
    return img_gris
            

In [ ]:
rdi = get_mouse_click(img_gris.astype('uint8'))
height,width=img_gris.shape
zona_delete = compute_mask(width,height,rdi.points)
img_mask = convine_mask(zona_delete,img_gris)

print np.min(zona_delete)

for i in xrange(len(img_mask)):
    c=1
    for j in xrange(len(img_mask[0])):
        if img_mask[i][j]==-100:
            img_mask[i][j]=img_mask[i-j][j]
            c=c+2
fig = plt.figure(124) 
plt.imshow(img_mask)
plt.title('img_mask')
plt.show()

In [37]:
print rdi.points

[(121.97580645161291, 96.572580645161281), (115.9274193548387, 182.25806451612902), (140.12096774193549, 207.45967741935485), (148.18548387096774, 228.62903225806451), (174.39516129032256, 236.69354838709677), (245.9677419354839, 170.16129032258064), (244.95967741935488, 106.65322580645164), (154.23387096774195, 62.298387096774206), (123.99193548387096, 93.548387096774206), (121.97580645161291, 96.572580645161281)]


In [51]:
print len(img_gris[0])

500
